# LOGISTIC REGRESSION MODEL CLASSIFICATION MOVIEW REVIEW
----- HO HUY HOA, NGUYEN MINH TRI----- 
## About Dataset:
1. id           22500 non-null
2. review       22500 non-null 
3. sentiment    22500 non-null 
## Model predict text classify was built with LogisticRegression
- Accuracy score nearly 88%
- Dont need more time to train model
## This model predict the text review is negative or positive with accuracy is 88%


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
data = pd.read_csv('./movie_review.csv', sep = '\t', encoding="latin-1")

In [3]:
data.head()

,id,review,sentiment
0,5814_8,With all this stuff going down at the moment w...,1
1,2381_9,"\The Classic War of the Worlds\"" by Timothy Hi...",1
2,7759_3,The film starts with a manager (Nicholas Bell)...,0
3,3630_4,It must be assumed that those who praised this...,0
4,9495_8,Superbly trashy and wondrously unpretentious 8...,1


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22500 entries, 0 to 22499
Data columns (total 3 columns):
id           22500 non-null object
review       22500 non-null object
sentiment    22500 non-null int64
dtypes: int64(1), object(2)
memory usage: 527.4+ KB


In [5]:
data.describe()

,sentiment
count,22500.000000
mean,0.501244
std,0.500010
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [26]:
data[data.isnull()].count()

id           0
review       0
sentiment    0
dtype: int64

In [35]:
data.drop(columns='id')

,review,sentiment
0,With all this stuff going down at the moment w...,1
1,"\The Classic War of the Worlds\"" by Timothy Hi...",1
2,The film starts with a manager (Nicholas Bell)...,0
3,It must be assumed that those who praised this...,0
4,Superbly trashy and wondrously unpretentious 8...,1
5,I dont know why people think this is such a ba...,1
6,"This movie could have been very good, but come...",0
7,"A friend of mine bought this film for ÃÂ£1, a...",0
8,<br /><br />This movie is full of references. ...,1
9,"What happens when an army of wetbacks, towelhe...",0


In [53]:
print('A number of negative review: ', len(data[data['sentiment'] == 0]))
print('A number of positive review: ', len(data[data['sentiment'] == 1]))


A number of negative review:  11222
A number of positive review:  11278


In [7]:
# stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hoahh2201\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
# Removing special characters and "trash"
import re
def preprocessor(text):
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text)
    
    # Save emoticons for later appending
    # If we do not use the non-capturing pattern '?:' for the groups, the returned result is a list of tuples, each tuple represents for each group. 
    # If we use non-capturing grouping '(?:)',that is, we group but not capture them, we can return a string
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    
    # Remove any non-word character(including white spaces) and append the emoticons,
    # Removing the nose character for standarization. Convert to lower case
    # If we don't use Quantifier '+', we only search for one character (non-alphanumeric character)
    # What this Regex do is Selecting a sequence of non-alphanumeric characters
    # We use join() because the 'emoticons' is actually a list
    # We replace '-' with '' because ':-)' is the same as ':)'
    text = re.sub('([\W]+)(d|p)*', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    
    return text

preprocessor(':) Today is fine :-D :P')

' today is fine  :) :D :P'

In [9]:
# tokenizer and stemming
# tokenizer: to break down our twits in individual words
# stemming: reducing a word to its root 
from nltk.stem import PorterStemmer
porter = PorterStemmer()

def tokenizer_porter(text):
    stem_tokens=[]
    tokens = text.split(' ')
    for token in tokens:
        stem_tokens.append(porter.stem(token))
    return stem_tokens

In [23]:
# split the dataset in train and test
from sklearn.model_selection import train_test_split
X = data['review']
y = data['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=101)

In [24]:
# Using Pipeline to concat tfidf step and LogisticRegression step
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

# If we don't pass these params, defaultly they are None
# If we don't pass stop_words, and preprocessor, our data is not good enough
# If we don't pass tokenizer, TfidfVectorizer will use the default one, but here we have tokenizer combined with stemming, better than default
tfidf = TfidfVectorizer(stop_words=stop,
                        tokenizer=tokenizer_porter,
                        preprocessor=preprocessor)

clf = Pipeline([('vect', tfidf), ('clf', LogisticRegression(random_state=0))])
clf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function prepr...e, penalty='l2', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])

In [25]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
predictions = clf.predict(X_test)
print('accuracy:',accuracy_score(y_test,predictions))

accuracy: 0.8868148148148148


In [13]:
data_test = pd.read_csv('./movie_review_noLabel.csv', sep = '\t', encoding="latin-1")

In [14]:
data_test.head()

,id,review
0,10633_1,I watched this video at a friend's house. I'm ...
1,4489_1,`The Matrix' was an exciting summer blockbuste...
2,3304_10,This movie is one among the very few Indian mo...
3,3350_3,The script for this movie was probably found i...
4,1119_1,Even if this film was allegedly a joke in resp...


In [15]:
Test_X = data_test['review']


In [16]:
predictions_test = clf.predict(Test_X)


In [17]:
len(Test_X)

2500

In [18]:
len(predictions_test)

2500

In [19]:
type(predictions_test)

numpy.ndarray

In [20]:
data_test['setement'] = predictions_test

In [21]:
data_test.head()

,id,review,setement
0,10633_1,I watched this video at a friend's house. I'm ...,0
1,4489_1,`The Matrix' was an exciting summer blockbuste...,0
2,3304_10,This movie is one among the very few Indian mo...,1
3,3350_3,The script for this movie was probably found i...,0
4,1119_1,Even if this film was allegedly a joke in resp...,0


In [22]:
data_test.to_csv('moview_review_test')